# Threshold Joye-Libert Secure Aggregation

In [1]:
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS
from ftsa.protocols.buildingblocks.VectorEncoding import VES
import random

t = 412
threshold = 7
nclients = 10
drops = 3
keysize = 2048 
inputsize = 16
dimension = 10

users = list(range(1, nclients+1))
alive = list(range(1, nclients-drops+1))
dropped = list(range(nclients-drops+1,nclients+1))


VE = VES(keysize // 2, nclients, inputsize, dimension)
TJL = TJLS(nclients, threshold, VE)

print("users: ", users)
print("alive: ", alive)
print("dropped: ", dropped)

users:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alive:  [1, 2, 3, 4, 5, 6, 7]
dropped:  [8, 9, 10]


### Generate JL keys (**Setup**)

In [2]:
# generate keys
pp, skey, ukey = TJL.Setup(keysize)

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=12272...00921, H(x)=0xcc45)>
 - Aggregator Key: <ServerKey -0x1f1565b>
 - User Keys: {0: <UserKey 0xa8205483>, 1: <UserKey 0x145bf181>, 2: <UserKey 0x1750f5e6>, 3: <UserKey 0x3a6bde93>, 4: <UserKey 0x15bac97d>, 5: <UserKey 0x1784bed3>, 6: <UserKey 0x8bc0bfe3>, 7: <UserKey 0xbda2ed54>, 8: <UserKey 0xf8a089ec>, 9: <UserKey 0x6f621c01>}


### Share the JL user key with all other users (**SKShare**)


In [3]:

# share secret key of each user
shares = {}
for u in users: 
    shares[u] = TJL.SKShare(ukey[u-1],threshold, users)

# distribute the shares
from collections import defaultdict
usersshares = defaultdict(dict)
for v in users:
    for u in users: 
        usersshares[u][v] = shares[v][u-1]

### Protect inputs of alive users with the users' keys (**Protect**)

In [4]:
# generate $nclients random vectors
L = []
for _ in range(nclients):
    l =[]
    for i in range(dimension):
        l.append(random.randint(0,1000))
    L.append(l)

# encrypt all vectors
E=[]
for u in alive:
    E.append(TJL.Protect(pp, ukey[u-1], t, L[u-1]))

E

[[<EncryptedNumber 55891...85736>],
 [<EncryptedNumber 90137...91587>],
 [<EncryptedNumber 10825...08622>],
 [<EncryptedNumber 76636...09684>],
 [<EncryptedNumber 61184...92921>],
 [<EncryptedNumber 35969...93352>],
 [<EncryptedNumber 94455...77951>]]

### Protect the zero-value with the key shares of the dropped users  (**ShareProtect**)

In [5]:
if dropped != []:
    yzeroshare ={}
    for u in alive:
        dropped_users_shares = []
        for v in dropped:
            dropped_users_shares.append(usersshares[u][v])
        yzeroshare[u] = TJL.ShareProtect(pp, dropped_users_shares,t)


### Combine shares of the protected zero-value (**ShareCombine**)

In [6]:
if dropped != []:
    yzero = TJL.ShareCombine(pp, list(yzeroshare.values()), threshold)
else:
    yzero = None

### Aggregate all user protected inputs and the protected zero-value of dropped users (**Agg**)

In [7]:
S = TJL.Agg(pp, skey, t, E, yzero)


summ=L[0]
from operator import add
for l in L[1:nclients-drops]:
    summ = list(map(add, summ, l))

print(S)
# check if the result is correct
print("Verify: ", S == summ)

[5031, 3495, 5866, 3811, 2470, 2572, 3824, 3443, 3713, 4690]
Verify:  True
